In [ ]:
import pandas as pd
import numpy as np
from optbinning import BinningProcess


In [ ]:
df_master = pd.read_csv('merged_kpi_dataset_v2.csv')

bureau 집계 파일이랑 합치기

In [ ]:
# 집계된 bureau 파일 불러오기
agg_bureau = pd.read_csv("agg_bureau.csv")

# app 기준 ID 추출
valid_ids = set(df_master["SK_ID_CURR"])
agg_bureau_filtered = agg_bureau[agg_bureau["SK_ID_CURR"].isin(valid_ids)].copy()

# 겹치는 컬럼 확인
common_cols = list(set(agg_bureau_filtered.columns) & set(df_master.columns))
common_cols.remove("SK_ID_CURR")  # ID는 남겨야 함

# 덮어쓰기 위해 기존 컬럼 제거
if common_cols:
    df_master.drop(columns=common_cols, inplace=True)

# 병합 (덮어쓰기 처리됨)
df_master = df_master.merge(agg_bureau_filtered, on="SK_ID_CURR", how="left")

# 저장
df_master.to_csv("merged_kpi_dataset_v3.csv", index=False, encoding="utf-8-sig")
print("병합 완료 → 'merged_kpi_dataset_v3.csv'")

-------

### 데이터 수정

In [ ]:
merged = df_master.copy()

In [ ]:
agg_bureau

In [ ]:
# 세 컬럼 모두 0인 경우 1, 아니면 0 → 거래내역 없음 플래그
merged['NO_CREDIT_HISTORY_FLAG'] = (merged[cols].sum(axis=1) == 0).astype(int)

In [ ]:
merged['CC_90PLUS'] = (merged['CC_90PLUS'] > 0).astype(int)

--------

In [ ]:
merged.head()

### 참고용 ) 상관관계 확인

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 수치형 변수만 추출 (상관계수는 수치형 변수 간에만 의미 있음)
numeric_df = merged.select_dtypes(include=['number'])

# 2. 상관계수 계산
corr = numeric_df.corr()

# 3. 히트맵 그리기
plt.figure(figsize=(15, 13))
sns.heatmap(corr, annot=True, fmt=".1f", cmap='coolwarm', square=True, linewidths=0.5)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()


In [ ]:
merged.corr()['TARGET'].abs().sort_values(ascending=False)

In [ ]:
# 1. 상관계수 행렬에서 절댓값 기준으로 0.7 이상 & 자기 자신은 제외
high_corr = corr.abs()

# 2. 자기 자신 상관계수 제외 (1.0인 대각선 제거)
mask = np.triu(np.ones_like(high_corr, dtype=bool))
high_corr_filtered = high_corr.mask(mask)

# 3. 0.7 이상 필터링
high_corr_pairs = high_corr_filtered.stack()
high_corr_pairs = high_corr_pairs[high_corr_pairs >= 0.7].sort_values(ascending=False)

# 결과 출력
print("상관계수 절댓값이 0.7 이상인 변수쌍:")
print(high_corr_pairs)


### 결측률 분석

In [ ]:
df = merged.copy()

In [ ]:
df.isnull().mean().sort_values(ascending=False)

### WOE 계산하기 전 전처리 (결측치 처리 & binning)

In [ ]:
df.columns

In [ ]:
# 범주형 변수
cat_cols = ['IS_LONG_TERM_LOAN', 'IS_HIGH_INTEREST', 'HAS_HIGH_RISK_DEBT', 'NO_CREDIT_HISTORY_FLAG']

# 연속형 변수
cont_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'CC_90PLUS', 'AVG_SK_DPD_x', 'AVG_CARD_USAGE_MONTHS', 'AVG_UTIL_RATIO',
       'AVG_MIN_PAY_RATIO', 'AVG_REPAY_RATIO', 'CREDIT_CARD_HISTORY_YEARS',
       'INST_90PLUS', 'avg_payment_delay', 'pct_underpaid',
       'pct_full_payments', 'LOAN_HISTORY_YEARS', 'AVG_SK_DPD_y', 'LOAN_HISTORY_YEARS(prev)',
       'NUM_POS_APPLICATIONS', 'NUM_CASH_APPLICATIONS',
       'NUM_REVOLVING_APPLICATIONS', 'REFUSAL_RATE', 'IS_OVERDUE_LONG',
       'IS_OVERDUE_SHORT', 'IS_LONG_TERM_ACTIVE', 'CREDIT_INCREASED',
       'DELTA_CREDIT_SUM_IF_INCREASED', 'DELTA_CREDIT_PCT_IF_INCREASED',
       'OVERLAP', 'OVERLAP_YEARS', 'END_STATUS_SCORE',
       'bureau_IS_ACTIVE_ESTIMATED', 'bureau_IS_CLOSED_ACTUAL',
       'bureau_DAYS_CREDIT_ENDDATE',
]

In [ ]:
len(cat_cols + cont_cols)

In [ ]:
def calc_woe(df, feature, target='TARGET', bins=10):
    import numpy as np
    import pandas as pd

    df_tmp = df[[feature, target]].copy()

    is_binary = pd.api.types.is_numeric_dtype(df[feature]) and df[feature].nunique(dropna=False) <= 2

    if pd.api.types.is_numeric_dtype(df_tmp[feature]):
        # 결측은 일단 그대로 두고 구간화 (qcut) 시 결측 제외
        df_tmp['bin'] = pd.qcut(df_tmp[feature], bins, duplicates='drop')
        
        # 'bin'이 categorical 타입이라 'Missing' 범주 추가
        if 'Missing' not in df_tmp['bin'].cat.categories:
            df_tmp['bin'] = df_tmp['bin'].cat.add_categories('Missing')

        # 결측인 행에 'Missing' 할당
        df_tmp.loc[df_tmp[feature].isna(), 'bin'] = 'Missing'

        group = df_tmp.groupby('bin')[target]

    else:
        # 범주형 변수인 경우 결측값을 'Missing'으로 대체
        df_tmp[feature] = df_tmp[feature].fillna('Missing')
        group = df_tmp.groupby(feature)[target]

    good = group.sum() + 0.5
    bad = group.count() - group.sum() + 0.5
    dist_good = good / good.sum()
    dist_bad = bad / bad.sum()
    woe = np.log(dist_good / dist_bad)
    woe_df = pd.DataFrame({'WOE': woe})
    return woe_df




In [ ]:
woe_results = {}

for feature in cont_cols + cat_cols:
    woe_df = calc_woe(df, feature, target='TARGET', bins=10)
    woe_results[feature] = woe_df


In [ ]:
i = 1
for feature, woe_df in woe_results.items():
    print(f"=== {i}. {feature} ===")
    i += 1
    print(woe_df)
    print()


In [ ]:
import matplotlib.pyplot as plt

def plot_woe(woe_df, title='WOE Plot'):
    woe_df = woe_df.reset_index()
    plt.figure(figsize=(10, 6))
    plt.bar(woe_df.iloc[:, 0].astype(str), woe_df['WOE'], edgecolor='black')
    plt.xlabel('Bin')
    plt.ylabel('WOE')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# 사용 예시: EXT_SOURCE_1 변수 시각화
plot_woe(woe_results['EXT_SOURCE_1'], title='WOE for EXT_SOURCE_1')


In [ ]:
import matplotlib.pyplot as plt

def plot_woe(woe_df, title='WOE Plot'):
    woe_df = woe_df.reset_index()
    plt.figure(figsize=(10, 6))
    plt.bar(woe_df.iloc[:, 0].astype(str), woe_df['WOE'], edgecolor='black')
    plt.xlabel('Bin')
    plt.ylabel('WOE')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# 사용 예시: EXT_SOURCE_1 변수 시각화
plot_woe(woe_results['LOAN_HISTORY_YEARS(prev)'], title='WOE forLOAN_HISTORY_YEARS(prev)')


### IV 계산

In [ ]:
def calc_iv_from_woe_df(woe_df):
    """
    WOE DataFrame에서 IV 계산.
    WOE DF는 반드시 WOE 값만 포함되어야 하며,
    그룹별 good/bad 분포 정보가 필요합니다.
    """
    iv_df = woe_df.copy()
    iv_df['dist_good'] = np.exp(iv_df['WOE']) / (np.exp(iv_df['WOE']) + 1)
    iv_df['dist_bad'] = 1 / (np.exp(iv_df['WOE']) + 1)
    iv_df['IV'] = (iv_df['dist_good'] - iv_df['dist_bad']) * iv_df['WOE']
    return iv_df['IV'].sum()


In [ ]:
iv_results = {}

for feature, woe_df in woe_results.items():
    iv = calc_iv_from_woe_df(woe_df)
    iv_results[feature] = iv

# 결과를 Series로 정리 후 내림차순 정렬
iv_series = pd.Series(iv_results).sort_values(ascending=False)

# 결과 출력
pd.set_option('display.max_rows', None)  # 전체 출력
print(iv_series)
